Load some libraries

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

Provide input paths to data files (might need to be changed)

In [9]:
data_hosts_path = '../kaggle/input/olympic-games-medals-19862018/olympic_hosts.csv'
data_medals_path = '../kaggle/input/olympic-games-medals-19862018/olympic_medals.csv'
data_results_path = '../kaggle/input/olympic-games-medals-19862018/olympic_results.csv'
data_athletes_path = '../kaggle/input/olympic-games-medals-19862018/olympic_athletes.csv'

Load the hosts dataset (general data about each olympics game)

In [6]:
df_hosts = pd.read_csv(data_hosts_path)
df_hosts.head()

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


Load the medals dataset (who won what medals, and in what olympics)

In [8]:
df_medals = pd.read_csv(data_medals_path)
df_medals.head()

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE


Let's merge medals and hosts data to see how data can be merged

In [36]:
data = df_medals.merge(df_hosts, how='left', left_on='slug_game', right_on='game_slug')
data.loc[0]

discipline_title                                                   Curling
slug_game                                                     beijing-2022
event_title                                                  Mixed Doubles
event_gender                                                         Mixed
medal_type                                                            GOLD
participant_type                                                  GameTeam
participant_title                                                    Italy
athlete_url              https://olympics.com/en/athletes/stefania-cons...
athlete_full_name                                     Stefania CONSTANTINI
country_name                                                         Italy
country_code                                                            IT
country_3_letter_code                                                  ITA
game_slug                                                     beijing-2022
game_end_date            

Medals for Athletics

In [11]:
data_medal = pd.concat([data_team_a, data_team_b, data_athlete], axis=0).reset_index(drop=True)

NameError: name 'data_team_a' is not defined

In [ ]:
data_summer = data_medal[data_medal['game_season']=='Summer'].reset_index(drop=True)

data_athletics = 